# Differnt Models on both proccesed and preprocessed

In [1]:
!echo "y" | conda install -c huggingface transformers -y
!pip install huggingface-hub

Retrieving notices: ...working... done
Channels:
 - huggingface
 - pytorch
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.

DEPRECATION: Loading egg at /Users/ching/miniconda3/envs/myenv/lib/python3.11/site-packages/huggingface_hub-0.19.0-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
# Install packages using Conda
!echo "y" |conda install pandas

Channels:
 - pytorch
 - defaults
 - huggingface
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.



In [3]:
!pip install torch 
!pip install chardet

DEPRECATION: Loading egg at /Users/ching/miniconda3/envs/myenv/lib/python3.11/site-packages/huggingface_hub-0.19.0-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/ching/miniconda3/envs/myenv/lib/python3.11/site-packages/huggingface_hub-0.19.0-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [4]:
!pip install openpyxl

DEPRECATION: Loading egg at /Users/ching/miniconda3/envs/myenv/lib/python3.11/site-packages/huggingface_hub-0.19.0-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


## 1st LiYuan/amazon-review-sentiment-analysis

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

# Load the pre-trained model and tokenizer
model_name = "LiYuan/amazon-review-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load your preprocessed data from the CSV file
df = pd.read_csv('preprocessed_data_veryless.csv')

# Function to predict sentiment using the model
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    sentiment_score = torch.softmax(logits, dim=1).tolist()[0]  # Get the sentiment score
    return predicted_class, max(sentiment_score)  # Keep only the maximum sentiment score

# Apply the sentiment prediction to the 'Preprocessed_Body' column
df[['Sentiment_Label', 'Max_Sentiment_Score']] = df['Preprocessed_Body'].apply(predict_sentiment).apply(pd.Series)

# Display the DataFrame with the new 'Sentiment_Label' and 'Max_Sentiment_Score' columns
print(df[['Preprocessed_Body', 'Sentiment_Label', 'Max_Sentiment_Score']])

# Save the DataFrame with sentiment labels and scores to a new CSV file
df.to_csv('1_sentiment_analysis_results_processed_less.csv', index=False)


                                     Preprocessed_Body  Sentiment_Label  \
0    5 stars nice little bluetooth box bought for a...              4.0   
1    5 stars easy to operate .. the product was ama...              4.0   
2    5 stars love my jteman cell phone holder i sim...              4.0   
3    5 stars great buy on sale solid easy to use an...              4.0   
4    5 stars this is a gift he loves listening to m...              4.0   
..                                                 ...              ...   
370  1 stars chinese spy device my wife got me this...              0.0   
371  1 stars received as a gift dead out of the box...              0.0   
372  1 stars horrible im beyond late on this review...              0.0   
373  1 stars defective audio issue i got this as a ...              0.0   
374  1 stars will not charge i thought this was sup...              0.0   

     Max_Sentiment_Score  
0               0.975006  
1               0.987549  
2               0.

## 2nd nlptown/bert-base-multilingual-uncased-sentiment

In [6]:
import torch
import openpyxl
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

# Load the pre-trained model and tokenizer
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load your preprocessed data from the CSV file
df = pd.read_csv('preprocessed_data_veryless.csv')

# Function to predict sentiment using the model
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    sentiment_score = torch.softmax(logits, dim=1).max().item()  # Get the highest sentiment score
    return predicted_class, sentiment_score

# Apply the sentiment prediction to the 'Preprocessed_Body' column
df[['Sentiment_Label', 'Sentiment_Score']] = df['Preprocessed_Body'].apply(predict_sentiment).apply(pd.Series)

# Display the DataFrame with the new 'Sentiment_Label' and 'Sentiment_Score' columns
print(df[['Preprocessed_Body', 'Sentiment_Label', 'Sentiment_Score']])

# Save the DataFrame with sentiment labels and scores to a new CSV file
df.to_csv('2_bert-base-multilingual-uncased-sentiment_processed_less.csv', index=False)


                                     Preprocessed_Body  Sentiment_Label  \
0    5 stars nice little bluetooth box bought for a...              4.0   
1    5 stars easy to operate .. the product was ama...              4.0   
2    5 stars love my jteman cell phone holder i sim...              4.0   
3    5 stars great buy on sale solid easy to use an...              4.0   
4    5 stars this is a gift he loves listening to m...              4.0   
..                                                 ...              ...   
370  1 stars chinese spy device my wife got me this...              0.0   
371  1 stars received as a gift dead out of the box...              0.0   
372  1 stars horrible im beyond late on this review...              0.0   
373  1 stars defective audio issue i got this as a ...              0.0   
374  1 stars will not charge i thought this was sup...              0.0   

     Sentiment_Score  
0           0.889987  
1           0.973634  
2           0.985940  
3      

## 3rd lxyuan/distilbert-base-multilingual-cased-sentiments-student

In [7]:
from transformers import pipeline
import pandas as pd

# Load the sentiment analysis pipeline with the specified model
classifier_distilbert = pipeline("sentiment-analysis", model="lxyuan/distilbert-base-multilingual-cased-sentiments-student")

# Load your CSV file into a DataFrame (replace 'your_file.csv' with your actual file path)
df = pd.read_csv('preprocessed_data_veryless.csv')

# Create empty lists to store labels and scores
labels = []
scores = []

# Process each review and store the results
for index, row in df.iterrows():
    text = row['Preprocessed_Body']  # Assuming 'Body' is the column containing the reviews
    try:
        # Truncate the text to the maximum sequence length
        truncated_text = text[:512]
        result = classifier_distilbert(truncated_text)[0]  # Assuming you only want the first result if multiple are returned
        labels.append(result['label'])
        scores.append(result['score'])
    except Exception as e:
        print(f"Skipped review at index {index} due to error: {e}")
        labels.append('unknown')
        scores.append('unknown')

# Add new columns to the DataFrame
df['Sentiment_Label'] = labels
df['Sentiment_Score'] = scores

# Save the processed DataFrame to a new CSV file
df.to_csv('3_lxyuan_distilbert_processed_less.csv', index=False)


## 4th sohan-ai/sentiment-analysis-model-amazon-reviews

In [8]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load the fine-tuned model from Hugging Face
model_name = "sohan-ai/sentiment-analysis-model-amazon-reviews"
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained(model_name)

# Load your CSV file into a DataFrame (replace 'your_file.csv' with your actual file path)
df = pd.read_csv('preprocessed_data_veryless.csv')

# Create empty lists to store labels and scores
labels = []
scores = []

# Process each review and store the results
for index, row in df.iterrows():
    text = row['Preprocessed_Body']  # Assuming 'Body' is the column containing the reviews
    try:
        inputs = tokenizer(text, return_tensors="pt")
        outputs = model(**inputs)
        predicted_label = "positive" if outputs.logits.argmax().item() == 1 else "negative"
        labels.append(predicted_label)
        scores.append(outputs.logits.softmax(dim=1).max().item())
    except Exception as e:
        print(f"Skipped review at index {index} due to error: {e}")
        labels.append('unknown')
        scores.append('unknown')

# Add new columns to the DataFrame
df['Sentiment_Label'] = labels
df['Sentiment_Score'] = scores

# Save the processed DataFrame to a new CSV file
df.to_csv('4_sohan_ai_sentiment_analysis_processed_less.csv', index=False)


Token indices sequence length is longer than the specified maximum sequence length for this model (689 > 512). Running this sequence through the model will result in indexing errors


Skipped review at index 101 due to error: The size of tensor a (689) must match the size of tensor b (512) at non-singleton dimension 1
Skipped review at index 116 due to error: The size of tensor a (559) must match the size of tensor b (512) at non-singleton dimension 1


In [9]:
import pandas as pd
! echo "y" | conda install -c conda-forge xlsxwriter

# Read CSV files into DataFrames
df1 = pd.read_csv('1_sentiment_analysis_results_processed_less.csv')
df2 = pd.read_csv('2_bert-base-multilingual-uncased-sentiment_processed_less.csv')
df3 = pd.read_csv('3_lxyuan_distilbert_processed_less.csv')
df4 = pd.read_csv('4_sohan_ai_sentiment_analysis_processed_less.csv')

# Create an Excel writer object
with pd.ExcelWriter('4_models_results_veryless.xlsx', engine='xlsxwriter') as writer:
    # Write each DataFrame to a different sheet
    df1.to_excel(writer, sheet_name='Model_1', index=False)
    df2.to_excel(writer, sheet_name='Model_2', index=False)
    df3.to_excel(writer, sheet_name='Model_3', index=False)
    df4.to_excel(writer, sheet_name='Model_4', index=False)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Channels:
 - conda-forge
 - pytorch
 - defaults
 - huggingface
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /Users/ching/miniconda3/envs/myenv

  added / updated specs:
    - xlsxwriter


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2023.7.22          |     pyhd8ed1ab_0         150 KB  conda-forge
    openssl-3.1.4              |       h0d3ecfb_0         2.0 MB  conda-forge
    xlsxwriter-3.1.9           |     pyhd8ed1ab_0         118 KB  conda-forge
    ------------------------------------------------------------